In [1]:
import os
import pandas as pd
from skimage import transform
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold
from sklearn import metrics
from skimage import color

from Multiview_autoencoder.Multiview_autoencoder import MultiviewImageDataset, MultiViewConvAutoencoder, extract_features
from CV_CNN_classifier.CV_CNN_classifier import FeatsDataset, CNNClassifier, evaluate

In [3]:
# Load device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [15]:
# Load CV images of reaction mixture, substrate and catalyst
image_tensors = []
image_arrays = []
rgb_imgs = []
gray_imgs = []
number = []
image_folder = './MultiviewImageDataset/cv-mixture' 
for filename in os.listdir(image_folder):
    if filename.endswith('.png'):
        num = filename.split('.')[0]
        num = int(num) - 1
        number.append(num)
        img = Image.open(os.path.join(image_folder, filename))
        img_array = np.array(img)
        if len(img_array.shape) == 3:
            #print(num)
            rgb_image = color.rgba2rgb(img)
            rgb_imgs.append(rgb_image)
            gray_img = color.rgb2gray(rgb_image)
            gray_imgs.append(gray_img)
            img_array = np.array(gray_img)
        image_arrays.append(img_array)
        new_size = (400, 400)
        resized_img = transform.resize(img_array, new_size)
        img_tensor = torch.tensor(resized_img)
        img_tensor = img_tensor.to(torch.float32)
        image_tensors.append(img_tensor)

image_tensors_sub = []
image_arrays_sub = []
rgb_imgs_sub = []
gray_imgs_sub = []
number_sub = []
sub_image_folder = './MultiviewImageDataset/cv-sub' 
for filename in os.listdir(sub_image_folder):
    if filename.endswith('.png'):
        num = filename.split('-')[0]
        num = int(num) - 1
        number_sub.append(num)
        img = Image.open(os.path.join(sub_image_folder, filename))
        img_array = np.array(img)
        if len(img_array.shape) == 3:
            #print(num)
            rgb_image = color.rgba2rgb(img)
            rgb_imgs_sub.append(rgb_image)
            gray_img = color.rgb2gray(rgb_image)
            gray_imgs_sub.append(gray_img)
            img_array = np.array(gray_img)
        image_arrays_sub.append(img_array)
        new_size = (400, 400)
        resized_img = transform.resize(img_array, new_size)
        img_tensor = torch.tensor(resized_img)
        img_tensor = img_tensor.to(torch.float32)
        image_tensors_sub.append(img_tensor)

sub_dict = {}
for idx, sub in zip(number_sub,image_tensors_sub):
    sub_dict[idx] = sub
sub_train = []
for i in number:
    y = sub_dict[i]
    sub_train.append(y)        

image_tensors_cat = []
image_arrays_cat = []
rgb_imgs_cat = []
gray_imgs_cat = []
number_cat = []
cat_image_folder = './MultiviewImageDataset/cv-cat'
for filename in os.listdir(cat_image_folder):
    if filename.endswith('.png'):
        num = filename.split('-')[0]
        num = int(num) - 1
        number_cat.append(num)
        img = Image.open(os.path.join(cat_image_folder, filename))
        img_array = np.array(img)
        if len(img_array.shape) == 3:
            #print(num)
            rgb_image = color.rgba2rgb(img)
            rgb_imgs_cat.append(rgb_image)
            gray_img = color.rgb2gray(rgb_image)
            gray_imgs_cat.append(gray_img)
            img_array = np.array(gray_img)
        image_arrays_sub.append(img_array)
        new_size = (400, 400)
        resized_img = transform.resize(img_array, new_size)
        img_tensor = torch.tensor(resized_img)
        img_tensor = img_tensor.to(torch.float32)
        image_tensors_cat.append(img_tensor)

cat_dict = {}
for idx, cat in zip(number_cat,image_tensors_cat):
    cat_dict[idx] = cat
cat_train = []
for i in number:
    y = cat_dict[i]
    cat_train.append(y)

In [7]:
# Load learning target
df = pd.read_csv('./MultiviewImageDataset/data_all.csv')
target = df['tag'].to_list()
target = np.array(target)

num_csv = df['num'].to_list()
num_csv = np.array(num_csv)

target_dict = {}
for idx, y in zip(num_csv, target):
    target_dict[idx] = y

In [9]:
# Training autoencoder
dataset = MultiviewImageDataset(image_tensors, sub_train, cat_train, number)
data_loader = DataLoader(dataset, batch_size=1, shuffle=True)    
model = MultiViewConvAutoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())
model = model.to(device)

num_train = []
decoding_list1 = []
decoding_list2 = []
decoding_list3 = []
for epoch in range(30):
    stop_training = False
    for img1, img2, img3, num in data_loader:
        img1, img2, img3 = img1.to(device), img2.to(device), img3.to(device)
        if epoch == 29:
            num_train.append(num)       
        optimizer.zero_grad()
        rec1, rec2, rec3, _ = model(img1, img2, img3)
        if epoch == 29:
            decoding_list1.append(rec1)
            decoding_list2.append(rec2)
            decoding_list3.append(rec3)
        img1 = img1.unsqueeze(1)
        img2 = img2.unsqueeze(1)
        img3 = img3.unsqueeze(1)
        loss1 = criterion(rec1, img1)
        loss2 = criterion(rec2, img2)
        loss3 = criterion(rec3, img3)
        loss_total = loss1 + loss2 + loss3 
        loss_total.backward()
        optimizer.step()
        if loss_total.item() < 0.0002:
            stop_training = True
            break 
    print(f'Autoencoder_epoch [{epoch+1}], Loss: {loss_total.item():.4f}')
    if stop_training:
        print(f"Early stopping at Autoencoder_epoch {epoch+1}, loss reached {loss_total.item():.6f}")
        break  

Autoencoder_epoch [1], Loss: 0.0042
Autoencoder_epoch [2], Loss: 0.0074
Autoencoder_epoch [3], Loss: 0.0019
Autoencoder_epoch [4], Loss: 0.0011
Autoencoder_epoch [5], Loss: 0.0002
Early stopping at Autoencoder_epoch 5, loss reached 0.000169


In [11]:
# Extract compressed features of the autoencoder
shared_features, num_list = extract_features(model, data_loader, device)
shared_features = shared_features.cpu()

target_AE = []
for i in num_list:
    i = i.numpy()
    y = target_dict[(i[0])]
    target_AE.append(y)    
target_AE = np.array(target_AE)
dataset_f = FeatsDataset(shared_features, num_list, target_AE)

In [13]:
# Training CNN classifier
classifier = CNNClassifier()
classifier = classifier.to(device)
criterion_c = nn.CrossEntropyLoss()
optimizer_c = optim.Adam(classifier.parameters(), lr=0.001)
kf = KFold(n_splits=10, shuffle=True,random_state=0)

all_acc_list = []
all_loss_list = []
acc_train_list = []
fold_num = 1
for train_indices, val_indices in kf.split(dataset_f):
        
    train_dataset = [dataset_f[i] for i in train_indices]
    val_dataset = [dataset_f[i] for i in val_indices]

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True) 
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)
    
    predicted_val_list = []
    predicted_val_list = np.array(predicted_val_list)
    labels_val_list = []
    num_val_list = []
    acc_val_list= []
    loss_list = []
    
    for epoch in range(100):
        model.train()
        for image, num, label in train_loader:
            image = image.to(device, dtype=torch.float32, non_blocking=True)
            label = label.to(device, non_blocking=True)
            optimizer_c.zero_grad()
            output = classifier(image)
            label = label.view(-1)
            loss = criterion_c(output, label)
            loss.backward()
            optimizer_c.step()
        label_val, prediction_val, num_val = evaluate(classifier, val_loader, device)
        label_train, prediction_train, num_train = evaluate(classifier, train_loader, device)
        label_train = np.array(label_train)
        acc_train = metrics.accuracy_score(label_train, prediction_train)
        acc_train_list.append(acc_train)
        label_val = np.array(label_val)
        acc_val = metrics.accuracy_score(label_val, prediction_val)
        acc_val_list.append(acc_val)
        loss = loss.detach()
        loss_list.append(loss)
    
    all_acc_list.append(max(acc_val_list))
    all_loss_list.append(loss_list[-1].cpu().item())
    print(
        "Fold {:02d}, Validation Acc. {:.4f}, Loss {:.4f} ".format(
        fold_num, acc_val_list[-1], loss_list[-1])
        )
    fold_num += 1
average_accuracy = np.mean(all_acc_list)
average_loss = np.mean(all_loss_list)

print("Accuracy_all {:.4f}, Loss {:.4f}".format(average_accuracy, average_loss))

Fold 01, Validation Acc. 0.8000, Loss 0.0125 
Fold 02, Validation Acc. 1.0000, Loss 0.0027 
Fold 03, Validation Acc. 1.0000, Loss 0.0324 
Fold 04, Validation Acc. 1.0000, Loss 0.0000 
Fold 05, Validation Acc. 1.0000, Loss 0.0332 
Fold 06, Validation Acc. 1.0000, Loss 0.0257 
Fold 07, Validation Acc. 1.0000, Loss 0.0035 
Fold 08, Validation Acc. 0.8000, Loss 0.0108 
Fold 09, Validation Acc. 1.0000, Loss 0.0009 
Fold 10, Validation Acc. 1.0000, Loss 0.0273 
Accuracy_all 0.9800, Loss 0.0149
